Попытка преобразовать датасет под yolo формат. К сожалению, из-за структуры самого датасета (сильного разброса по количеству изображений на каждый класс) результат обучения моделей не особо выдающийся.

In [ ]:
! pip install -q kaggle
! pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00


In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"svyatoslavivanov","key":"209967f757c7c1ad863d12c6a454f18c"}'}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! kaggle datasets download -d watchman/rtsd-dataset

  2% 282M/17.1G [00:02<01:49, 166MB/s]

In [ ]:
! mkdir /content/traffic-dataset
! unzip rtsd-dataset.zip -d train

In [ ]:
! rm rtsd-dataset.zip

In [ ]:
! rm -rf /content/traffic-dataset-yolo/train/
! rm -rf /content/traffic-dataset-yolo/validation/
! mkdir /content/traffic-dataset-yolo/
! mkdir /content/traffic-dataset-yolo/train/
! mkdir /content/traffic-dataset-yolo/validation/
! mkdir /content/traffic-dataset-yolo/train/images/
! mkdir /content/traffic-dataset-yolo/train/labels/
! mkdir /content/traffic-dataset-yolo/validation/images/
! mkdir /content/traffic-dataset-yolo/validation/labels/

mkdir: cannot create directory ‘/content/traffic-dataset-yolo/’: File exists


In [ ]:
import json
import cv2
import os
import matplotlib.pyplot as plt
import shutil
import random
import wandb

In [ ]:
# define the paths
input_path = "/content/traffic-dataset/"
output_path = "/content/traffic-dataset-yolo/"

In [ ]:
classes_array = set()

def convert_to_yolo_format(json_file, out_labels_folder, out_images_folder, limit):
    with open(json_file) as f:
        data = json.load(f)

    images = {image['id']: image for image in data['images']}
    annotations = data['annotations']
    categories = {category['id']: category['name'] for category in data['categories']}

    # Write class names to a separate file
    with open(output_path + 'classes.txt', 'w') as f:
        for _, name in sorted(categories.items()):
            classes_array.add(name)

    counter = 0

    for annotation in annotations:
        image_id = annotation['image_id']
        image_info = images[image_id]

        # Calculate YOLO format
        x_min, y_min, bbox_width, bbox_height = annotation['bbox']
        center_x = (x_min + bbox_width / 2) / image_info['width']
        center_y = (y_min + bbox_height / 2) / image_info['height']
        width = bbox_width / image_info['width']
        height = bbox_height / image_info['height']

        # Prepare the line to write
        line = f"{annotation['category_id']} {center_x} {center_y} {width} {height}\n"

        # Copy image file
        src_image_path = os.path.join(input_path + "rtsd-frames/rtsd-frames/", image_info['file_name'].split('/')[1])
        dst_image_path = os.path.join(out_images_folder, image_info['file_name'].split('/')[1])
        if not os.path.exists(dst_image_path):
            shutil.copy(src_image_path, dst_image_path)
            print(f"File {image_info['file_name'].split('/')[1]} copied successfully.")

        # Write to file
        output_file = f"{out_labels_folder}/{image_info['file_name'].split('/')[1].replace('.jpg', '.txt')}"
        with open(output_file, 'a') as file:
            file.write(line)

        counter += 1

        if counter == limit:
          break

train_labels_folder = '/content/traffic-dataset-yolo/train/labels/'
train_images_folder = '/content/traffic-dataset-yolo/train/images/'

validation_labels_folder = '/content/traffic-dataset-yolo/validation/labels/'
validation_images_folder = '/content/traffic-dataset-yolo/validation/images/'

# Example usage
convert_to_yolo_format(input_path + 'val_anno.json', validation_labels_folder, validation_images_folder, 500)
convert_to_yolo_format(input_path + 'train_anno.json', train_labels_folder, train_images_folder, 3000)

# Write class names to a separate file
with open(output_path + 'classes.txt', 'w') as f:
    for name in list(classes_array):
        f.write(name + '\n')

In [ ]:
import yaml

# Define your data for the YAML file
data = {
    'train': [train_images_folder, train_labels_folder],
    'val': [validation_images_folder, validation_labels_folder],
    'nc': len(classes_array),  # replace with your actual number of classes
    'names': list(classes_array)  # replace with your class names
}

# Specify the output YAML file name
output_yaml_file = 'config.yaml'

# Writing to the YAML file
with open(f'/content/traffic-dataset-yolo/{output_yaml_file}', 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

print(f"YAML file '{output_yaml_file}' created successfully.")


YAML file 'config.yaml' created successfully.


In [ ]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 3.2e-05,
    "architecture": "YOLO8s",
    "dataset": "RUSSIAN_TRAFFIC_SIGNS_DATASET",
    "epochs": 10,
    }
)

In [ ]:
!pip install ultralytics

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO('yolov8s.yaml')


                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

In [ ]:
model.train(data=f'/content/traffic-dataset-yolo/{output_yaml_file}', epochs=100, batch=8, close_mosaic=1)

Ultralytics YOLOv8.0.215 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/content/traffic-dataset-yolo/config.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, 

train: Scanning /content/traffic-dataset-yolo/train/labels... 6066 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6066/6066 [00:10<00:00, 561.65it/s] 


train: New cache created: /content/traffic-dataset-yolo/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/traffic-dataset-yolo/validation/labels... 5000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5000/5000 [00:02<00:00, 1721.41it/s]


val: New cache created: /content/traffic-dataset-yolo/validation/labels.cache
Plotting labels to runs/detect/train8/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=3.2e-05, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train8
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10      2.92G      2.716      57.51      2.025         16        640:  72%|███████▏  | 547/759 [03:01<01:10,  3.02it/s]


KeyboardInterrupt: ignored

In [ ]:
wandb.finish()
metrics = model.val(split="val")

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"